# Importing Required Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Load the Dataset

In [ ]:
dataset = pd.read_csv('cnn_news.csv')

# Preprocessing the Data
### Removing Punctuation, Numbers and making all the text lowercase

In [ ]:
dataset['headlines'] = dataset['headlines'].str.replace("[^a-zA-Z#]", " ")
dataset['headlines'] = dataset['headlines'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
dataset['headlines'] = dataset['headlines'].apply(lambda x: x.lower())

# Tokenizing the Text

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['headlines'])
word_index = tokenizer.word_index

# Converting Text to Sequences and Padding them

In [ ]:
sequences = tokenizer.texts_to_sequences(dataset['headlines'])
padded = pad_sequences(sequences, padding='post')

# Splitting the Dataset into Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded, dataset['category'], test_size=0.2, random_state=0)

# Building the Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=padded.shape[1]),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiling the Model

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Training the Model

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluating the Model

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Testing the Model

In [ ]:
prediction = model.predict(X_test)
prediction = [round(pred[0]) for pred in prediction]

# Confusion Matrix and Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, prediction)
cr = classification_report(y_test, prediction)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", cr)